# Parameter estimation using qualitative data

This Notebook eplains the use of qualitative data for parameter estimation, as described in [Schmiester et al. (2019)](https://www.biorxiv.org/content/10.1101/848648v1.abstract). An example model is provided in `example_qualitative`

## Import model and create pyPESTO objective

In [ ]:
import pypesto
import pypesto.petab
import pypesto.optimize as optimize

import amici
import petab
import numpy as np
from pypesto.hierarchical.optimal_scaling_solver import OptimalScalingInnerSolver
import pypesto.logging
import logging

In [ ]:
petab_folder = './example_qualitative/'
yaml_file = 'example_qualitative.yaml'

petab_problem = petab.Problem.from_yaml(petab_folder + yaml_file)

importer = pypesto.petab.PetabImporter(petab_problem)

model = importer.create_model()

In [ ]:
# To allow for hierarchical optimization, set hierarchical=True, when creating the objective

objective = importer.create_objective(hierarchical=True)
problem = importer.create_problem(objective)

engine = pypesto.engine.SingleCoreEngine()

# pypesto.logging.log_to_console(logging.INFO)

optimizer = optimize.ScipyOptimizer(method='Nelder-Mead',
                                   options={'disp': True, 'maxiter': 500, 'maxfev': 500, 'fatol': 1e-10})

n_starts = 10

## Run optimization using optimal scaling approach

Different options can be used for the optimal scaling approach:
- method: `standard` / `reduced`
- reparameterized: `True` / `False`
- intervalConstraints: `max` / `max-min`
- minGap: Any float value

It is recommended to use the reduced method with reparameterization as it is the most efficient and robust choice.

When no options are provided, the default is the reduced and reparameterized formulation with max as interval constraint and `minGap=1e-10`.

### Run optimization using the reduced and reparameterized approach

In [ ]:
problem.objective.calculator.inner_solver = OptimalScalingInnerSolver(options={'method': 'reduced',
                                                                               'reparameterized': True,
                                                                               'intervalConstraints': 'max',
                                                                               'minGap': 1e-10})

res_reduced_reparameterized = optimize.minimize(problem, n_starts=n_starts, optimizer=optimizer, engine=engine)

### Run optimization using the reduced non-reparameterized approach

In [ ]:
problem.objective.calculator.inner_solver = OptimalScalingInnerSolver(options={'method': 'reduced',
                                                                               'reparameterized': False,
                                                                               'intervalConstraints': 'max',
                                                                               'minGap': 1e-10})

res_reduced = optimize.minimize(problem, n_starts=n_starts, optimizer=optimizer, engine=engine)

### Run optimization using the standard approach

In [ ]:
problem.objective.calculator.inner_solver = OptimalScalingInnerSolver(options={'method': 'standard',
                                                                               'reparameterized': False,
                                                                               'intervalConstraints': 'max',
                                                                               'minGap': 1e-10})

res_standard = optimize.minimize(problem, n_starts=n_starts, optimizer=optimizer, engine=engine)

### Compare results

Reduced formulation leads to improved computation times

In [ ]:
time_standard = res_standard.optimize_result.get_for_key('time')
print(f"Mean computation time for standard approach: {np.mean(time_standard)}")

time_reduced = res_reduced.optimize_result.get_for_key('time')
print(f"Mean computation time for reduced approach: {np.mean(time_reduced)}")

time_reduced_reparameterized = res_reduced_reparameterized.optimize_result.get_for_key('time')
print(f"Mean computation time for reduced reparameterized approach: {np.mean(time_reduced_reparameterized)}")

All approaches yield the same objective function values

In [ ]:
from pypesto.visualize import waterfall

waterfall([res_standard, res_reduced, res_reduced_reparameterized])